<a href="https://colab.research.google.com/github/RoboTuan/ML4IOT_HMW/blob/main/HMW2/HW2_ex2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 pip install -q tensorflow-model-optimization

     |████████████████████████████████| 174kB 9.9MB/s 


In [2]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import os
from zipfile import ZipFile
import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.python.core.api.sparsity import keras as sparsity
import zlib



In [3]:
if not os.path.isdir('./HMW'):
  !git clone "https://github.com/RoboTuan/ML4IOT_HMW.git"  
  !mv 'ML4IOT_HMW' 'HMW'

ROOT_DIR = 'HMW/HMW2/'
tflite_models=ROOT_DIR + "tflite_models/"

Cloning into 'ML4IOT_HMW'...
remote: Enumerating objects: 187, done.
remote: Counting objects: 100% (187/187), done.
remote: Compressing objects: 100% (160/160), done.
remote: Total 187 (delta 106), reused 67 (delta 21), pack-reused 0
Receiving objects: 100% (187/187), 1.69 MiB | 5.47 MiB/s, done.
Resolving deltas: 100% (106/106), done.


In [4]:
#parser = argparse.ArgumentParser()
#parser.add_argument('--version', type=str, required=True, help='a,b or c')
#parser.add_argument('--mfcc', action='store_true', help='use MFCCs')
#args = parser.parse_args()

model_type='c' #args.version
mfcc=True
alpha=0.75

In [5]:
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

In [6]:
zip_path = tf.keras.utils.get_file(
        origin="http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip",
        fname='mini_speech_commands.zip',
        extract=True,
        cache_dir='.', cache_subdir='data')
data_dir = os.path.join('.', 'data', 'mini_speech_commands')
filenames = tf.io.gfile.glob(str(data_dir) + '/*/*')
filenames = tf.random.shuffle(filenames)
num_samples = len(filenames)
total = 8000

182083584/182082353 [==============================] - 3s 0us/step


In [7]:
train_files = tf.strings.split(tf.io.read_file(ROOT_DIR +'kws_train_split.txt'),sep='\n')[:-1]
val_files= tf.strings.split(tf.io.read_file(ROOT_DIR +'kws_val_split.txt'),sep='\n')[:-1]
test_files = tf.strings.split(tf.io.read_file(ROOT_DIR +'kws_test_split.txt'),sep='\n')[:-1]

 


In [8]:
LABELS = np.array(tf.io.gfile.listdir(str(data_dir)))
LABELS = LABELS[LABELS != 'README.md']

In [9]:
class SignalGenerator:
    def __init__(self, labels, sampling_rate, frame_length, frame_step,
            num_mel_bins=None, lower_frequency=None, upper_frequency=None,
            num_coefficients=None, mfcc=False):
        self.labels = labels
        self.sampling_rate = sampling_rate
        self.frame_length = frame_length
        self.frame_step = frame_step
        self.num_mel_bins = num_mel_bins
        self.lower_frequency = lower_frequency
        self.upper_frequency = upper_frequency
        self.num_coefficients = num_coefficients
        num_spectrogram_bins = (frame_length) // 2 + 1

        if mfcc is True:
            self.linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
                    self.num_mel_bins, num_spectrogram_bins, self.sampling_rate,
                    self.lower_frequency, self.upper_frequency)
            self.preprocess = self.preprocess_with_mfcc
        else:
            self.preprocess = self.preprocess_with_stft

    def read(self, file_path):
        parts = tf.strings.split(file_path, os.path.sep)
        label = parts[-2]
        label_id = tf.argmax(label == self.labels)
        audio_binary = tf.io.read_file(file_path)
        audio, _ = tf.audio.decode_wav(audio_binary)
        audio = tf.squeeze(audio, axis=1)

        return audio, label_id

    def pad(self, audio):
        zero_padding = tf.zeros([self.sampling_rate] - tf.shape(audio), dtype=tf.float32)
        audio = tf.concat([audio, zero_padding], 0)
        audio.set_shape([self.sampling_rate])

        return audio
    def get_spectrogram(self, audio):
        stft = tf.signal.stft(audio, frame_length=self.frame_length,
                frame_step=self.frame_step, fft_length=self.frame_length)
        spectrogram = tf.abs(stft)

        return spectrogram

    def get_mfccs(self, spectrogram):
        mel_spectrogram = tf.tensordot(spectrogram,
                self.linear_to_mel_weight_matrix, 1)
        log_mel_spectrogram = tf.math.log(mel_spectrogram + 1.e-6)
        mfccs = tf.signal.mfccs_from_log_mel_spectrograms(log_mel_spectrogram)
        mfccs = mfccs[..., :self.num_coefficients]

        return mfccs

    def preprocess_with_stft(self, file_path):
        audio, label = self.read(file_path)
        audio = self.pad(audio)
        spectrogram = self.get_spectrogram(audio)
        spectrogram = tf.expand_dims(spectrogram, -1)
        spectrogram = tf.image.resize(spectrogram, [32, 32])

        return spectrogram, label

    def preprocess_with_mfcc(self, file_path):
        audio, label = self.read(file_path)
        audio = self.pad(audio)
        spectrogram = self.get_spectrogram(audio)
        mfccs = self.get_mfccs(spectrogram)
        mfccs = tf.expand_dims(mfccs, -1)

        return mfccs, label

    def make_dataset(self, files, train):
        ds = tf.data.Dataset.from_tensor_slices(files)
        ds = ds.map(self.preprocess, num_parallel_calls=4)
        ds = ds.batch(32)
        ds = ds.cache()
        if train is True:
            ds = ds.shuffle(100, reshuffle_each_iteration=True)

        return ds
    



In [10]:
STFT_OPTIONS = {'frame_length': 256, 'frame_step': 128, 'mfcc': False}
MFCC_OPTIONS = {'frame_length': 640, 'frame_step': 320, 'mfcc': True,
        'lower_frequency': 20, 'upper_frequency': 4000, 'num_mel_bins': 40,
        'num_coefficients': 10}

In [11]:
#if args.mfcc is True:
if mfcc is True:
    options = MFCC_OPTIONS
    strides = [2, 1]
else:
    options = STFT_OPTIONS
    strides = [2, 2]

In [12]:
generator = SignalGenerator(LABELS, 16000, **options)
train_ds = generator.make_dataset(train_files, True)
val_ds = generator.make_dataset(val_files, False)
test_ds = generator.make_dataset(test_files, False)
units=8

In [13]:
dataset_dir= ROOT_DIR + "dataset"

tf.data.experimental.save(train_ds, f'{dataset_dir}/train_ds')
tf.data.experimental.save(val_ds, f'{dataset_dir}/val_ds')
tf.data.experimental.save(test_ds, f'{dataset_dir}/test_ds')

In [14]:
!zip -r test_ds.zip {dataset_dir + "/test_ds"}


  adding: HMW/HMW2/dataset/test_ds/ (stored 0%)
  adding: HMW/HMW2/dataset/test_ds/snapshot.metadata (stored 0%)
  adding: HMW/HMW2/dataset/test_ds/15031723494189575379/ (stored 0%)
  adding: HMW/HMW2/dataset/test_ds/15031723494189575379/00000000.shard/ (stored 0%)
  adding: HMW/HMW2/dataset/test_ds/15031723494189575379/00000000.shard/00000000.snapshot (deflated 8%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Model

##Model A

In [15]:
#MLP Model definition
mlp_model = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dense(units=8) #if silence 10
])

##Model B

In [16]:
#CNN_2D Model definition

cnn_model = keras.Sequential([
    keras.layers.Conv2D(filters=128,kernel_size=[3,3],strides=strides, use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.Conv2D(filters=128,kernel_size=[3,3],strides=strides, use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.Conv2D(filters=128,kernel_size=[3,3],strides=strides, use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(units=8) #10 if silence
])

##Model C

In [17]:
ds_cnn_model = keras.Sequential([
    keras.layers.Conv2D(filters=(alpha*256),kernel_size=[3,3],strides=strides, use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.DepthwiseConv2D(kernel_size=[3,3],strides=[1, 1], use_bias=False),
    keras.layers.Conv2D(filters=(alpha*256),kernel_size=[1,1],strides=[1,1], use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.DepthwiseConv2D(kernel_size=[3,3],strides=[1, 1], use_bias=False),
    keras.layers.Conv2D(filters=(alpha*256),kernel_size=[1,1],strides=[1,1], use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(units=8) #10 if silence else 8
])

##Model definition

In [18]:
if model_type=='a':
  model=mlp_model
  if (mfcc==True):
    path= './callback_val_chkp/mfcc_model_a'
  else:
    path= './callback_val_chkp/stft_model_a'

if model_type=='b':
  model=cnn_model
  if (mfcc==True):
    path= './callback_val_chkp/mfcc_model_b'
  else:
    path= './callback_val_chkp/stft_model_b'

if model_type=='c':
  model=ds_cnn_model
  if (mfcc==True):
    path= './callback_val_chkp/mfcc_model_c'
  else:
    path= './callback_val_chkp/stft_model_c'

In [19]:

cp_callback = keras.callbacks.ModelCheckpoint(
    filepath=path,
    # './callback_test_chkp/chkp_best',
    monitor='val_accuracy',
    verbose=0, 
    save_best_only=True,
    # save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch'
)

##Training

In [20]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,optimizer='adam', metrics=['accuracy'])
model.fit(train_ds, epochs=20, validation_data=val_ds,callbacks=[cp_callback])

Epoch 1/20
200/200 [==============================] - ETA: 0s - loss: 1.2934 - accuracy: 0.6072WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/tracking/tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./callback_val_chkp/mfcc_model_c/assets
200/200 [==============================] - 17s 83ms/step - loss: 1.2934 - accuracy: 0.6072 - val_loss: 0.8255 - val_accuracy: 0.7513
Epoch 2/20
200/200 [==============================] - 17s 84ms/step - loss: 0.6756 - accuracy: 0.8141 - val_loss: 0.5210 - val_accuracy: 0.8375
Epoch 3/20
200/200 [==============================] - 17s 84ms/step - loss: 0.473

##Validation

In [21]:
#model evaluation on validation set
val_accuracy= model.evaluate(val_ds)

25/25 [==============================] - 0s 17ms/step - loss: 0.2481 - accuracy: 0.9262


##Testing

In [22]:
test_accuracy= model.evaluate(test_ds)

25/25 [==============================] - 1s 20ms/step - loss: 0.2676 - accuracy: 0.9150


#Pruning

In [23]:
prune_low_magnitude=tfmot.sparsity.keras.prune_low_magnitude

pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,final_sparsity=0.90,begin_step=len(train_ds)*5,end_step=len(train_ds)*15)
}


model_for_pruning = prune_low_magnitude(model, **pruning_params)


Instructions for updating:
Please use `layer.add_weight` method instead.


In [24]:
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [25]:
callbacks = [tfmot.sparsity.keras.UpdatePruningStep()]


In [26]:
model_for_pruning.fit(train_ds,epochs=20,validation_data=val_ds,callbacks=callbacks) 

Epoch 1/20
200/200 [==============================] - 16s 78ms/step - loss: 0.0701 - accuracy: 0.9808 - val_loss: 0.2714 - val_accuracy: 0.9150
Epoch 2/20
200/200 [==============================] - 15s 77ms/step - loss: 0.0606 - accuracy: 0.9833 - val_loss: 0.2649 - val_accuracy: 0.9237
Epoch 3/20
200/200 [==============================] - 15s 77ms/step - loss: 0.0511 - accuracy: 0.9875 - val_loss: 0.3672 - val_accuracy: 0.8963
Epoch 4/20
200/200 [==============================] - 15s 76ms/step - loss: 0.0743 - accuracy: 0.9791 - val_loss: 0.2704 - val_accuracy: 0.9187
Epoch 5/20
200/200 [==============================] - 15s 77ms/step - loss: 0.0479 - accuracy: 0.9880 - val_loss: 0.2418 - val_accuracy: 0.9287
Epoch 6/20
200/200 [==============================] - 15s 76ms/step - loss: 0.1353 - accuracy: 0.9589 - val_loss: 0.2720 - val_accuracy: 0.9175
Epoch 7/20
200/200 [==============================] - 15s 76ms/step - loss: 0.1529 - accuracy: 0.9558 - val_loss: 0.2621 - val_accuracy:

In [27]:
model_for_pruning_accuracy = model_for_pruning.evaluate(
  test_ds, verbose=0)
 
print('Pruned test accuracy:', model_for_pruning_accuracy)

Pruned test accuracy: [0.2909991443157196, 0.9075000286102295]


In [28]:
model=tfmot.sparsity.keras.strip_pruning(model_for_pruning)

##tflite conversion

In [29]:
ROOT_DIR="./callback_val_chkp/"
tflite_models= ROOT_DIR + "tfliteModels/"
tflite_models_2= ROOT_DIR + "tfliteModelsNoZip/"
!mkdir ./callback_val_chkp/tfliteModels
!mkdir ./callback_val_chkp/tfliteModelsNoZip

if model_type=='a':
  if (mfcc==True):
    saved_model_dir= ROOT_DIR +"mfcc_model_a"
    tflite_model_dir= tflite_models + "model_a"
    tflite_model_dir_noZip= tflite_models_2 + "model_a"
  else:
    saved_model_dir= ROOT_DIR +"stft_model_a"
    tflite_model_dir= tflite_models + "model_a"
    tflite_model_dir_noZip= tflite_models_2 + "model_a"

if model_type=='b':
  if (mfcc==True):
    saved_model_dir= ROOT_DIR +"mfcc_model_b"
    tflite_model_dir= tflite_models + "model_b"
    tflite_model_dir_noZip= tflite_models_2 + "model_b"
  else:
    saved_model_dir= ROOT_DIR +"stft_model_b"
    tflite_model_dir= tflite_models + "model_b"
    tflite_model_dir_noZip= tflite_models_2 + "model_b"

if model_type=='c':
  if (mfcc==True):
    saved_model_dir= ROOT_DIR +"mfcc_model_c"
    tflite_model_dir= tflite_models + "model_c"
    tflite_model_dir_noZip= tflite_models_2 + "model_c"
  else:
    saved_model_dir= ROOT_DIR +"stft_model_c"
    tflite_model_dir= tflite_models + "model_c"
    tflite_model_dir_noZip= tflite_models_2 + "model_c"




converter=tf.lite.TFLiteConverter.from_keras_model(model) #from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
#converter.representative_dataset= representative_data_gen
tflite_model=converter.convert()

with open(tflite_model_dir, "wb") as fp:
  tflite_compressed = zlib.compress(tflite_model)
  fp.write(tflite_compressed)

with open(tflite_model_dir_noZip, "wb") as fp:
  fp.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpkvee18gz/assets


#Quantization

In [30]:
'''quantize_model = tfmot.quantization.keras.quantize_model
q_aware_model = quantize_model(model)
q_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
q_aware_model.fit(train_ds, epochs=20, validation_data=val_ds,callbacks=[cp_callback])

model=q_aware_model'''

"quantize_model = tfmot.quantization.keras.quantize_model\nq_aware_model = quantize_model(model)\nq_aware_model.compile(optimizer='adam',\n              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),\n              metrics=['accuracy'])\nq_aware_model.fit(train_ds, epochs=20, validation_data=val_ds,callbacks=[cp_callback])\n\nmodel=q_aware_model"

##Size and latency measurement

##Size measurement

In [31]:
os.path.getsize(tflite_model_dir)

24431